In [ ]:
! pip install langchain
! pip install sentence_transformers
! pip install faiss-gpu

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("./all_content.txt") as f:
    text_file = f.read()

text_splitter = RecursiveCharacterTextSplitter(
                                      chunk_size=500,
                                      chunk_overlap=20)

chunks = text_splitter.split_text(text_file)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
vectorStore = FAISS.from_texts(chunks, embeddings)

In [ ]:
from langchain import HuggingFaceHub
import os


os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv('HUGGINGFACEHUB_API_TOKEN')
llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.1 ,"max_length":512})

In [ ]:
from langchain.chains import RetrievalQA
from langchain.schema import retriever

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorStore.as_retriever())

In [ ]:
query="tell me stories about recycle electrical waste?"
chain.run(query)



1. In 2016, Quito, Ecuador, implemented a program to create electricity and compost from organic waste. The city's waste management system was designed to separate organic waste from other waste, and the electricity produced was used to power public lighting and other infrastructure. The program was a success, and it was later expanded to other cities in Ecuador.

2. In 2016, the city of Houston, Texas, USA, launched a program to incentivize reuse of construction materials. The program, called the Reuse Warehouse, allows homeowners and businesses to donate unwanted materials to the city, which then repurposes them for public use. The program has been a success, and it has been expanded to other cities in the US.

3. In 2016, the city of Quito, Ecuador, implemented a program to create electricity and compost from organic waste. The city's waste management system was designed to separate organic waste from other waste, and the electricity produced was used to power public lighting and other infrastructure. The program was a success, and it was later expanded to other cities in Ecuador.